In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
apify_api_token = os.environ['APIFY_API_TOKEN']

#### Apify是一个网络抓取和数据提取的云平台，它提供了一个由一千多个现成的应用程序组成的生态系统，这些应用程序被称为Actors，用于各种网络抓取、爬行和数据提取用例。例如，你可以用它来提取谷歌搜索结果、Instagram和Facebook的个人资料、亚马逊或Shopify的产品、谷歌地图的评论等等。
#### 在本例中，我们将使用Website Content Crawler Actor，它可以深度抓取文档、知识库、帮助中心或博客等网站，并从网页中提取文本内容。然后我们将文档输入矢量索引，并从中回答问题。

In [ ]:
from langchain.document_loaders.base import Document
from langchain.indexes import VectorstoreIndexCreator
from langchain.utilities import ApifyWrapper

In [ ]:
apify = ApifyWrapper()

#### 然后运行Actor，等待它完成，并将其结果从Apify数据集获取到LangChain文档加载器中。
#### 注意，如果在Apify数据集中已经有一些结果，可以使用ApifyDatasetLoader直接加载它们，如本笔记本所示。在该笔记本中，您还可以找到dataset_mapping_function的解释，该函数用于将字段从Apify数据集记录映射到LangChain Document字段。

In [ ]:
loader = apify.call_actor(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls":[{"url": "https://python.langchain.com/en/latest/"}]},
    dataset_mapping_function= lambda item: Document(
        page_content=item['text'] or "", metadata={"source": item["url"]}
    ),
)

In [ ]:
# Initialize the vector index from the crawled documents:
index = VectorstoreIndexCreator().from_loaders([loader])

In [ ]:
# And finally, query the vector index:
query = "What is LangChain?"
result = index.query_with_sources(query)

In [ ]:
print(result["answer"])
print(result["sources"])